# Basic Panel for Use Case
- presentation Monday, Nov 4, 2019

In [ ]:
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import panel as pn
import panel.widgets as pnw
import zarr
import xarray as xr
import sys

import util_panel

dir_analysis = '/home/jovyan/local-climate-data-tool/Analysis/Phase1_ProcessData'
sys.path.insert(0, dir_analysis)
import analysis_parameters


## 1. Read in the dummy data

In [ ]:
# Change this directory to be wherever you put the dummy data
DATA_PATH = analysis_parameters.DIR_DUMMY_DATA
# Keys for reading in data files (.zarr)
THIS_EXPERIMENT_ID = ['historical','ssp126', 'ssp370','ssp245','ssp585']
# Keys for dictionary with observational data too
EXPERIMENT_KEYS = THIS_EXPERIMENT_ID.copy()
EXPERIMENT_KEYS.append('historical_obs')

In [ ]:
DICT_TS = util_panel.read_data()

## 2. Example plot data for Seattle
- Check that the data looks good for an example case

In [ ]:
# For Seattle: 47.6062, 237.67
# For Chicago: 41.8781, 272.3702
thislat = 47.6062
thislon = 237.67

In [ ]:
def plot_timeseries(lon,lat):        
    fig = plt.figure(figsize=(10,6))
    dict_timeseries = read_data()
    for i in range(0,len(EXPERIMENT_KEYS)):
        # Define experiment id
        experiment_id = EXPERIMENT_KEYS[i]
        
        # Read in data for experiment id
        ds = dict_timeseries[experiment_id]

        # Select that one location
        ds=ds.sel(lon=thislon, lat=thislat, method='nearest')
        ds_annual = ds.groupby('time.year').mean('time')

        # Define colors for each experiment
        colors=['black','blue','green','orange','red','gray']
        # Plot each model
        plt.plot(ds_annual['year'].values,ds_annual['mean'].values,label = experiment_id, color=colors[i])

    plt.legend()
    plt.xlabel('Year')
    plt.ylabel('Temperature')
    plt.show()
    

plot_timeseries(thislon,thislat)

## 3. Generate panel

In [ ]:
dict_timeseries = util_panel.read_data()

In [ ]:
def hvplot(df, **kwargs):
    '''Generates plot -- parameter of time_series function below'''
    colors=['black','blue','green','orange','red','gray']
    return (df[0].hvplot(alpha=1.0, color=colors[0], width=800, label=EXPERIMENT_KEYS[0], ylabel='Temperature (K)', ylim=(250,300)) * 
            df[1].hvplot(alpha=0.5, color=colors[1], width=800, label=EXPERIMENT_KEYS[1], ylabel='Temperature (K)', ylim=(250,300)) * 
            df[2].hvplot(alpha=0.5, color=colors[2], width=800, label=EXPERIMENT_KEYS[2], ylabel='Temperature (K)', ylim=(250,300)) * 
            df[3].hvplot(alpha=0.5, color=colors[3], width=800, label=EXPERIMENT_KEYS[3], ylabel='Temperature (K)', ylim=(250,300)) * 
            df[4].hvplot(alpha=0.5, color=colors[4], width=800, label=EXPERIMENT_KEYS[4], ylabel='Temperature (K)', ylim=(250,300)) * 
            df[5].hvplot(alpha=1.0, color=colors[5], width=800, label=EXPERIMENT_KEYS[5], ylabel='Temperature (K)', ylim=(250,300))
           )

def time_series(lat, lon, annual_mean=False, view_fn=hvplot):
    '''Generate interactive portions of longitude and latitude for data'''
    ds = util_panel.read_data()
    df = [None]*6
    for i,exp in enumerate(EXPERIMENT_KEYS):
        one_ds = ds[exp]
        data_to_plot = one_ds.sel(lat=lat, lon=lon, method='nearest')
        xs = data_to_plot['time'].values
        ys = data_to_plot['mean'].values
        df[i] = pd.DataFrame(dict(y=ys), index=xs)
    return view_fn(df, lat=lat, lon=lon)

time_series(thislat, thislon)

In [ ]:
pn.extension()
latitude = pnw.FloatSlider(name='longitude', value=0, start=-80, end=80)
longitude = pnw.FloatSlider(name='longitude', value=180, start=0, end=360)


@pn.depends(latitude.param.value, longitude.param.value)
def reactive_time_series(latitude, longitude):
    return time_series(latitude,longitude)
widgets = pn.Column("<br>\n# Local Climate Tool",latitude,longitude)
side_panel = pn.Row(widgets, reactive_time_series)

side_panel

In [ ]:
# side_panel.show()
side_panel.servable()